In [1]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook
#%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import random
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
import matplotlib
import pandas as pd

%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record
import seaborn as sns
import pandas as pd


In [4]:
param_file = '/home/axtr7550/Chromosome_project/cellbgnet/cellbgnet/utils/reference_files/reference_Axel_45deg_lower_photon.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

model = cellbgnet.model.CellBGModel(param)

model.init_recorder()

model.init_eval_data()

INITIATING CUDA IMPLEMENTATION
training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21,

/home/axtr7550/anaconda3/envs/compile_spline/lib/python3.11/site-packages/imageio/plugins/pillow.py:297: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(
Eval image generation: 100%|████████████████████| 40/40 [00:10<00:00,  3.84it/s]


eval images shape: (40, 1041, 1302) contain 26814 molecules,


In [95]:


model_path = Path('/home/axtr7550/Chromosome_project/real_images/for_axel/model_rotated_45_venus_long_run_700_range.pkl')

In [96]:

with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)
    

INITIATING CPU IMPLEMENTATION


In [7]:
chromo_model.simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.00625,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [8]:
chromo_model.evaluation_params['eval_imgs'].shape


(30, 1041, 1302)

In [9]:
chromo_model.evaluation_params = model.evaluation_params

In [10]:
chromo_model.evaluation_params['ground_truth']

[[1,
  1,
  56651.471252441406,
  6348.040580749512,
  139.60188627243042,
  816.3612484931946],
 [2,
  1,
  45192.51159667969,
  6523.388633728027,
  284.99656915664673,
  1175.4266023635864],
 [3,
  1,
  45796.34826660156,
  6493.693504333496,
  241.25202000141144,
  747.6065307855606],
 [4,
  1,
  50339.94903564453,
  6515.825004577637,
  168.46098005771637,
  740.7311797142029],
 [5,
  1,
  55716.263122558594,
  6491.56852722168,
  -70.27114927768707,
  935.4722499847412],
 [6,
  1,
  63067.420349121094,
  6496.940727233887,
  190.7106637954712,
  881.8232417106628],
 [7,
  1,
  47788.79821777344,
  6578.153533935547,
  233.35254192352295,
  775.5559086799622],
 [8,
  1,
  63323.41339111328,
  6554.028472900391,
  213.83066475391388,
  953.7558853626251],
 [9,
  1,
  64140.17822265625,
  6541.066856384277,
  112.77209967374802,
  713.2889628410339],
 [10,
  1,
  26783.753051757812,
  6622.311897277832,
  249.13586676120758,
  789.8297309875488],
 [11,
  1,
  68462.75756835938,
  66

In [11]:
plot_train_record(chromo_model)

In [12]:
iterations = [key for key, value in chromo_model.recorder['count_loss'].items()]
count_loss = [value for key, value in chromo_model.recorder['count_loss'].items()]
loc_loss = [value for key, value in chromo_model.recorder['loc_loss'].items()]
bg_loss = [value for key, value in chromo_model.recorder['bg_loss'].items()]
cross_entropy_loss = [value for key, value in chromo_model.recorder['P_locs_error'].items()]

In [13]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].plot(iterations, count_loss)
ax[0, 0].set_xlabel('iterations')
ax[0, 0].set_ylabel('count loss')
ax[0, 1].plot(iterations, loc_loss)
ax[0, 1].set_xlabel('iterations')
ax[0, 1].set_ylabel('localization loss')
ax[1, 0].plot(iterations, bg_loss)
ax[1, 0].set_xlabel('iterations')
ax[1, 0].set_ylabel('noise free PSF loss')
ax[1, 1].plot(iterations, cross_entropy_loss)
ax[1, 1].set_xlabel('iterations')
ax[1, 1].set_ylabel('cross entropy loss')
plt.show()

In [14]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assess, assemble_full_img_predictions, limited_matching

#### For simulated data

In [15]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [16]:
plt.figure()
plt.imshow(eval_img[0])
plt.colorbar()
plt.show()

In [17]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [18]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.1,
                                             nms_threshold=0.5, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [19]:
chromo_model.evaluation_params.keys()

dict_keys(['number_images', 'use_cell_masks', 'density_in_cells', 'cell_masks_dir', 'cell_masks_filetype', 'edt_noise_map_path', 'eval_size', 'molecules_per_img', 'margin_empty', 'candidate_threshold', 'nms_threshold', 'print_result', 'batch_size', 'padded_background', 'eval_filenames', 'eval_imgs', 'ground_truth', 'fov_size'])

In [20]:
len(preds_tmp)

22024

##### Getting values of predicted and true x, y, z, i to plot distributions

In [21]:
ground_truth = chromo_model.evaluation_params['ground_truth']


In [22]:
len(ground_truth)

26814

In [23]:
min_int = np.percentile(np.array(ground_truth)[:, -1], 25)

In [24]:
min_int

749.5273612439632

In [25]:
min_int = 700


In [26]:
# Not sure which is x and which is y
GT_df = pd.DataFrame(ground_truth,columns=['emitter_index', 'image_index', 'y_position', 'x_position', 'z', 'phot'] )
GT_df['phot']

0         816.361248
1        1175.426602
2         747.606531
3         740.731180
4         935.472250
            ...     
26809     714.710265
26810     782.530367
26811     743.514508
26812     862.831146
26813     768.158376
Name: phot, Length: 26814, dtype: float64

In [27]:
fig, ax = plt.subplots()

sns.scatterplot(data=GT_df, x='z', y='phot')

<Axes: xlabel='z', ylabel='phot'>

In [28]:
perf_dict, matches = limited_matching(ground_truth, preds_tmp, min_int=min_int, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=250,
                                      tolerance_ax=np.inf)

FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 26814 ,preds: 22024
Recall: 0.824
Precision: 0.998
Jaccard: 82.224
RMSE_lat: 61.421
RMSE_ax: 133.046
RMSE_vol: 146.539
Jaccard/RMSE: 1.339
Eff_lat: 36.058
Eff_ax: 31.143
Eff_3d: 33.601
FN: 4505.0 FP: 42.0


### Preds list has the following in order

    1. counter of the molecule 
    2. image number used to index into the number of the image in the prediction arrays
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset


In [29]:
df_predictions = pd.DataFrame(preds_tmp, columns=["counter of the molecule","image number", "x","y","z","phot", "prob_post_nms","x_sigma","y_sigma","z_sigma","phot_sigma", "x_offset","y_offset"])

In [30]:
fig, ax = plt.subplots()
sns.scatterplot(data=df_predictions, x="phot", y="z_sigma", alpha=0.1)

<Axes: xlabel='phot', ylabel='z_sigma'>

### Plot distributions of true vs predicted to check for any biases on the full scale

matches is a matrix where rows correspond to a single localization,
columns are as follows. There are 12 columns, I only know what the first 8 are (x and y are in large FOV nm converted)

    1. ground truth x 
    2. ground truth y
    3. ground truth z
    4. ground truth photon counts
    5. predicted x
    6. predicted y
    7. predicted z
    8. predicted photon counts
    10. **Probably** z_sigma

In [52]:
df_matches =pd.DataFrame(matches,columns= ['GT_x', 'GT_y','GT_z', 'GT_phot', 'pred_x','pred_y','pred_z', 'pred_phot', 'Prob_nms','z_sigma', '', ''])
df_matches['GT_phot'].hist()


<Axes: >

In [32]:
save_path = '/home/axtr7550/Chromosome_project/real_images/'

df_matches.to_csv(save_path + 'rotated_700_matches.csv', index=False)
df_predictions.to_csv(save_path + 'rotated_700_predictions.csv', index=False)
GT_df.to_csv(save_path + 'rotated_700_GT.csv', index=False)

In [50]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='GT_phot',hue='Prob_nms', palette='viridis', alpha=0.3)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [53]:
fig, ax = plt.subplots()
simulated_phot_hist, phot_bins = np.histogram(GT_df['phot'], bins=20)
matched_phot_GT_hist, _ = np.histogram(df_matches['GT_phot'], bins=20)

sns.histplot(data=GT_df, x='phot')
sns.histplot(data=df_matches, x='GT_phot')

recall_phot_hist = matched_phot_GT_hist/simulated_phot_hist
fig, ax = plt.subplots()
plt.plot(phot_bins[:-1],recall_phot_hist, 'o')
ax.set_title('Recall by photon count')
ax.set_xlabel('Photon count')
ax.set_ylabel('Recall')


/tmp/ipykernel_130890/4066394776.py:8: RuntimeWarning: invalid value encountered in divide
  recall_phot_hist = matched_phot_GT_hist/simulated_phot_hist


Text(0, 0.5, 'Recall')

In [62]:
fig, ax = plt.subplots()
simulated_z_hist, z_bins = np.histogram(GT_df['z'], bins=20)
matched_z_GT_hist, _ = np.histogram(df_matches['GT_z'], bins=20)

sns.histplot(data=GT_df, x='z')
sns.histplot(data=df_matches, x='GT_z')

recall_z_hist = matched_z_GT_hist/simulated_z_hist
fig, ax = plt.subplots()
plt.plot(z_bins[:-1],recall_z_hist, 'o')
ax.set_title('Recall by z')
ax.set_xlabel('z')
ax.set_ylabel('Recall')


Text(0, 0.5, 'Recall')

In [61]:
fig, ax = plt.subplots()
simulated_z_hist, z_bins = np.histogram(GT_df['z'], bins=20)
matched_z_pred_hist, _ = np.histogram(df_matches['pred_z'], bins=20)
df_preditions_filtered = df_predictions[df_predictions['prob_post_nms'] > 0.7]

sns.histplot(data=GT_df, x='z', label='Ground truth')
sns.histplot(data=df_matches, x='pred_z', label='Predicted')
sns.histplot(data=df_preditions_filtered, x='z', label='Predicted_all')

plt.legend()


ratio_pred_true_z_hist = matched_z_pred_hist/simulated_z_hist
fig, ax = plt.subplots()
plt.plot(z_bins[:-1],ratio_pred_true_z_hist, 'o')
ax.set_title('Ratio of predicted z to true z by z')
ax.set_xlabel('z')
ax.set_ylabel('Ratio')


Text(0, 0.5, 'Ratio')

In [44]:
fig, ax = plt.subplots()
samples = np.random.normal(loc=-200, scale=100.0, size= 100000)
sampled_hist, sampled_bins = np.histogram(samples, bins=z_bins)
ax.plot(sampled_bins[:-1], sampled_hist, 'k-', lw=2, label='frozen pdf')


In [46]:
fig, ax = plt.subplots()



ax.plot(z_bins[:-1],sampled_hist*ratio_pred_true_z_hist, 'o')
ax.plot(z_bins[:-1], sampled_hist, 'k-', lw=2, label='frozen pdf')


In [58]:
fig, ax = plt.subplots()
p = 0.8
df_preditions_filtered = df_predictions[df_predictions['prob_post_nms'] > p]
sns.histplot(data=df_preditions_filtered, x='z', label='Predictions')
sns.histplot(data=GT_df, x='z', label='Ground Truth')
ax.set_title(f'Predictions vs. Ground Truth, p > {p}')
plt.legend()



In [57]:
fig, ax = plt.subplots()
p = 0.8

df_preditions_filtered = df_predictions[df_predictions['prob_post_nms'] > p]
sns.histplot(data=df_preditions_filtered, x='phot', label='Predictions')
sns.histplot(data=GT_df, x='phot', label='Ground Truth')
ax.set_title(f'Predictions vs. Ground Truth, p > {p}')

plt.legend()


In [41]:
fig, ax = plt.subplots()
sns.histplot(data=df_predictions, x='phot', y = 'prob_post_nms')


<Axes: xlabel='phot', ylabel='prob_post_nms'>

,emitter_index,image_index,y_position,x_position,z,phot
0,1,1,35030.782776,6554.463882,-178.401172,2047.145963
1,2,1,48523.277588,6578.952942,355.572641,2008.168817
2,3,1,58537.427368,6601.703835,-283.023179,905.029535
3,4,1,59079.378357,6613.182678,401.785433,1238.538086
4,5,1,60663.808899,6640.758781,181.996703,2606.345952
...,...,...,...,...,...,...
3413,368,10,27132.070770,62197.850952,-242.133737,1465.012282
3414,369,10,30626.784821,62192.657776,402.321279,1833.265185
3415,370,10,49195.526123,62235.714722,-471.272767,1335.255325
3416,371,10,55426.456909,62173.737793,100.402713,823.142767


In [51]:
plt.hist(torch.distributions.exponential.Exponential(torch.ones(1000)*5).sample())


(array([512., 211., 122.,  70.,  39.,  22.,  12.,   2.,   6.,   4.]),
 array([9.24681735e-05, 1.21272497e-01, 2.42452517e-01, 3.63632560e-01,
        4.84812558e-01, 6.05992615e-01, 7.27172613e-01, 8.48352671e-01,
        9.69532669e-01, 1.09071267e+00, 1.21189272e+00]),
 <BarContainer object of 10 artists>)

In [42]:
plt.close('all')

In [42]:
from scipy.stats import linregress

for i in range(-500, 600, 100):
    lower_bound = i
    upper_bound = i + 200
    data_df= df_matches[(df_matches['z_sigma'] > 0) & (df_matches['pred_z'] > lower_bound)& (df_matches['pred_z'] < upper_bound)]
    jp = sns.jointplot(data=data_df, x="pred_z", y="GT_z", kind="reg")


    # Customize the scatterplot color
    scatter_color = 'lightblue'
    jp.ax_joint.collections[0].set_facecolor(scatter_color)
    jp.ax_joint.collections[0].set_edgecolor(scatter_color)

    # Customize the regression line color
    reg_line_color = 'red'
    jp.ax_joint.lines[0].set_color(reg_line_color)

    slope, intercept, r_value, p_value, std_err = linregress(data_df['pred_z'], data_df['GT_z'])

    # Compute the uncertainty bounds for the regression line
    x_vals = np.linspace(data_df['pred_z'].min(), data_df['pred_z'].max(), 100)
    y_pred = slope * x_vals + intercept
    y_upper = y_pred + 1.96 * std_err
    y_lower = y_pred - 1.96 * std_err

    # Plot the regression line with uncertainty bounds
    jp.ax_joint.plot(x_vals, y_pred, color='red', label='Regression Line')
    jp.ax_joint.fill_between(x_vals, y_lower, y_upper, color='lightcoral', alpha=0.3, label='95% CI')

    jp.ax_joint.set_ylim(-500, 500)

    # Show the legend
    jp.ax_joint.legend([f'r={r_value:.2f}', f'p={p_value:.3f}', f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}"])
    jp.fig.suptitle(f"Not rotated")


type: need at least one array to concatenate

In [84]:

plt.close('all')

In [41]:
lower_bound = 125
upper_bound = 175
phot_low = 700
phot_high = 1500 
data_df= df_matches[(df_matches['GT_phot'] >= phot_low) & (df_matches['GT_phot']<=phot_high) & (df_matches['pred_z'] > lower_bound)& (df_matches['pred_z'] < upper_bound)]
jp = sns.jointplot(data=data_df, x="pred_z", y="GT_z", kind="reg")


# Customize the scatterplot color
scatter_color = 'lightblue'
jp.ax_joint.collections[0].set_facecolor(scatter_color)
jp.ax_joint.collections[0].set_edgecolor(scatter_color)

# Customize the regression line color
reg_line_color = 'red'
jp.ax_joint.lines[0].set_color(reg_line_color)

slope, intercept, r_value, p_value, std_err = linregress(data_df['pred_z'], data_df['GT_z'])

# Compute the uncertainty bounds for the regression line
x_vals = np.linspace(data_df['pred_z'].min(), data_df['pred_z'].max(), 100)
y_pred = slope * x_vals + intercept
y_upper = y_pred + 1.96 * std_err
y_lower = y_pred - 1.96 * std_err

# Plot the regression line with uncertainty bounds
jp.ax_joint.plot(x_vals, y_pred, color='red', label='Regression Line')
jp.ax_joint.fill_between(x_vals, y_lower, y_upper, color='lightcoral', alpha=0.3, label='95% CI')
jp.ax_joint.errorbar(data_df["pred_z"], data_df["GT_z"], yerr=data_df['z_sigma']*1.96, fmt='o', color='lightblue', ecolor='gray', capsize=3)
jp.ax_joint.set_ylim([-500, 500])
# Show the legend
jp.ax_joint.legend([f'r={r_value:.2f}', f'p={p_value:.3f}', f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}"])
jp.fig.suptitle(f'True position of predicted dots with {lower_bound}<z<{upper_bound}, {phot_low}<photon count<{phot_high}')  # y=1.02 adjusts the title position slightly above the plot


type: name 'linregress' is not defined

In [ ]:
sampled_z = [item[4] for item in ground_truth]
sampled_ph = [item[5] for item in ground_truth]

In [51]:
gt_x, pred_x = matches[:, 0], matches[:, 4]
residual_x = (gt_x - pred_x)
gt_y, pred_y = matches[:, 1], matches[:, 5]
residual_y = (gt_y - pred_y)
gt_z, pred_z = matches[:, 2], matches[:, 6]
residual_z = (gt_z - pred_z)
gt_ph, pred_ph = matches[:, 3], matches[:, 7]
residual_ph = (gt_ph, pred_ph)

In [52]:
matplotlib.style.use('ggplot')


In [87]:
fig = plt.figure()

sns.scatterplot(data=df_matches, x="pred_z", y='pred_phot', hue='GT_z',size='GT_phot',  palette='coolwarm', alpha=0.5)
#sns.scatterplot(data=df, x="GT_z", y='GT_phot')


<Axes: xlabel='pred_z', ylabel='pred_phot'>

In [88]:

fig = plt.figure()

sns.scatterplot(data=df_matches, x="GT_z", y='GT_phot', hue='pred_z',size='pred_phot',  palette='coolwarm', alpha=0.5)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [149]:

def plot_subset(df, ax, z_low, z_high, phot_low, phot_high):
    data_df = df[(df['GT_z'] <=z_high)&(df['GT_z']>=z_low) & (df['GT_phot'] >= phot_low) & (df['GT_phot']<=phot_high)].reset_index()

    sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot', alpha=0.3, label=f'True positions {z_low}<z<{z_high} and {phot_low}<phot<{phot_high}')
    sns.scatterplot(ax=ax, data=data_df, x="pred_z", y='pred_phot',alpha=0.8, c='red', label='Corresponding predicted positions')

    z_start = data_df.loc[:,'GT_z']
    phot_start = data_df.loc[:, 'GT_phot']
    dz = data_df.loc[:, 'pred_z']-data_df.loc[:, 'GT_z']
    dphot = data_df.loc[:, 'pred_phot']-data_df.loc[:, 'GT_phot' ]

    #for i in range(len(data_df)):
     #   ax.arrow(z_start[i], phot_start[i], dz[i], dphot[i] ,width=0.01, facecolor='white', edgecolor='white', alpha=0.5)

    return ax

fig, ax = plt.subplots(figsize=(10,10))

sns.scatterplot(ax = ax, data=df_matches, x="pred_z", y='pred_phot',  palette='coolwarm', alpha=0.5, c='green', label='All predicted positions')

ax.set_xlabel("Z")
ax.set_ylabel("Photon count")

z_low = 400
z_high = 500
phot_low = 700  
phot_high = 2000

ax = plot_subset(df_matches, ax, z_low, z_high, phot_low, phot_high)
#ax = plot_subset(df, ax, -350, -300, 0, 2000)


/tmp/ipykernel_5067/2802139919.py:19: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.scatterplot(ax = ax, data=df_matches, x="pred_z", y='pred_phot',  palette='coolwarm', alpha=0.5, c='green', label='All predicted positions')


In [91]:

def plot_subset_flipped(df, ax, z_low, z_high, phot_low, phot_high):
    data_df = df[(df['pred_z'] <=z_high)&(df['pred_z']>=z_low) & (df['pred_phot'] >= phot_low) & (df['pred_phot']<=phot_high)].reset_index()

    sns.scatterplot(ax=ax, data=data_df, x="pred_z", y='pred_phot', alpha=0.3, label=f'Predicted positions {z_low}<z<{z_high} and {phot_low}<phot<{phot_high}')
    sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot',  palette='coolwarm', alpha=0.8, c='red', label=f'Corresponding GT positons')

    return ax

fig, ax = plt.subplots(figsize=(10,10))

sns.scatterplot(ax = ax, data=df_matches, x="GT_z", y='GT_phot',   alpha=0.5, c='green', label=f'All true positions')

ax.set_xlabel("Z")
ax.set_ylabel("Photon count")

z_low = 150
z_high = 250
phot_low = 700  
phot_high = 1200

ax.set_title(f"Spread of true values of predictions between {z_low}nm and {z_high}nm")


ax = plot_subset_flipped(df_matches, ax, z_low, z_high, phot_low, phot_high)
#ax = plot_subset(df, ax, -350, -300, 0, 2000)


/tmp/ipykernel_5067/2059549628.py:5: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot',  palette='coolwarm', alpha=0.8, c='red', label=f'Corresponding GT positons')


In [214]:
df_matches['z_error'] =  df_matches['pred_z'] - df_matches['GT_z']
df_matches['phot_error'] = df_matches['pred_phot'] - df_matches['GT_phot']
df_matches['abs_z_error'] = abs(df_matches['z_error'])

fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='z_error')
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='phot_error')


<Axes: xlabel='GT_z', ylabel='phot_error'>

In [43]:
def generate_cat_columns(df_matches, n_bins, z_bounds = [-500, 500], phot_bounds = [700, 3000]):

    z_bins = np.linspace(z_bounds[0], z_bounds[1], n_bins)
    phot_bins = np.linspace(phot_bounds[0],phot_bounds[1], n_bins)
    prob_bins = np.linspace(0.1,1.3, n_bins)


    df_matches['z_cat'] = None
    df_matches['phot_cat'] = None
    
    df_matches['GT_z_cat'] = None
    df_matches['GT_phot_cat'] = None


    df_matches['prob_cat'] = None


    for i in range(len(z_bins)-1):
        df_matches.loc[(df_matches['pred_z'] >= z_bins[i]) & (df_matches['pred_z'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
        df_matches.loc[(df_matches['pred_phot'] >= phot_bins[i]) & (df_matches['pred_phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]
        
        df_matches.loc[(df_matches['GT_z'] >= z_bins[i]) & (df_matches['GT_z'] <= z_bins[i+1]),'GT_z_cat'] = z_bins[i]
        df_matches.loc[(df_matches['GT_phot'] >= phot_bins[i]) & (df_matches['GT_phot'] <= phot_bins[i+1]),'GT_phot_cat'] = phot_bins[i]

        df_matches.loc[(df_matches['Prob_nms'] >= prob_bins[i]) & (df_matches['Prob_nms'] <= prob_bins[i+1]),'prob_cat'] = prob_bins[i]
    
    return df_matches, z_bins, phot_bins


In [ ]:

sns.histplot(data=df_matches, x="pred_z", y='GT_z', bins=100)



In [216]:
sns.histplot(data=df_matches, x="Prob_nms", y='pred_z',hue='prob_cat', hue_order = prob_bins,bins=50)
#df_matches = df_matches[df_matches['Prob_nms'] < 0.5]


<Axes: xlabel='pred_z', ylabel='GT_z'>

In [44]:
labels_dict = {'z_cat':'Z (nm)', 'phot_cat': 'Photon count','GT_z_cat':'GT Z (nm)', 'GT_phot_cat': 'GT Photon count', 'z_error': 'z error (nm)', 'prob_cat':'Predicted probability', \
    'phot_error': 'photon count error', 'abs_z_error': 'absolute Z-error (nm)', 'abs_phot_error': 'absolute Photon count error', \
        'GT_z': 'ground truth Z (nm)', 'GT_phot': 'ground truth photon count', 'pred_z': 'predicted Z (nm)', 'pred_phot': 'predicted photon count'}

In [45]:
def plot_heatmap_mean(df, x_col, y_col, val_col, precision=0, extra_title='', count=False, x_bins=False, y_bins=False):



    if count:
        means = df.groupby([x_col, y_col]).count()[val_col].unstack(fill_value=0)
    else:
        means = df.groupby([x_col, y_col]).mean()[val_col].unstack(fill_value=0)

    zero_df = pd.DataFrame(index=x_bins, columns=y_bins)
    zero_df.fillna(0, inplace=True)
    zero_df.update(means)

    means = zero_df

    means = means.stack()
    means = means.sort_index()
    
    means_np = means.to_numpy()# Maybe add more bins and make sure axis are correct
    side_length = int(np.sqrt(means_np.shape[0]))
    
    fig, ax = plt.subplots()
    ar = np.array(means.index).reshape((side_length,side_length))
    x_axis_vals = np.array([val[0] for val in ar[:,0]])
    step = x_axis_vals[1] - x_axis_vals[0]
    x_axis_vals = (x_axis_vals + step//2).astype(int)


    y_axis_vals = np.array([val[1] for val in ar[0,::-1]])
    step = y_axis_vals[1] - y_axis_vals[0]
    y_axis_vals = (y_axis_vals - step//2).astype(int)

    data = np.flip(means_np.reshape((side_length,side_length)).T, axis=0)
    sns.heatmap(data, xticklabels=x_axis_vals, 
                yticklabels=y_axis_vals, annot=True, fmt=f".{precision}f", mask=(data==0))
    ax.set_xlabel(labels_dict[x_col])
    ax.set_ylabel(labels_dict[y_col])
    fig.suptitle(f"Mean {labels_dict[val_col]} for different photon counts and Z-positions \n {extra_title}")

    return data, fig, ax 


In [58]:
df_matches_cols, z_bins, phot_bins = generate_cat_columns(df_matches, 15)

df_data = df_matches_cols.copy()
#df_data = df_data[df_data['GT_phot']<1700]

plot_heatmap_mean(df_data, 'z_cat', 'phot_cat', 'GT_z', x_bins=z_bins, y_bins=phot_bins)


/tmp/ipykernel_124709/1015808203.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = df.groupby([x_col, y_col]).mean()[val_col].unstack(fill_value=0)


(array([[   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,    0.        ,  105.90159521,
          154.22005455,  183.09143484,  203.52996886,  204.97874916,
          152.00195462,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,   65.00039995,  113.30884571,
          183.13881904,  195.24374604,  210.90483878,    0.        ,
          203.96857709,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ,
            0.        ,    0.        ,  151.10337521,  117.73683596,
          180.26056729,  211.90004498,  259.22117233,  294.14676502,
       

In [56]:
plt.close('all')

In [346]:
df_matches['prob_cat']

0        0.7
1       None
2       0.85
3       0.85
4       None
        ... 
9565     0.4
9566     0.4
9567     0.7
9568     0.4
9569     0.4
Name: prob_cat, Length: 9570, dtype: object

In [347]:
plt.close('all')

In [ ]:


phot_bins = list(range(700, 3100, 230))
phot_bins


In [209]:
plt.close('all')

In [98]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='GT_phot', hue='z_error', palette='magma', hue_norm=(100, 300))

type: Could not interpret value `z_error` for parameter `hue`

In [235]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x="GT_z", y='GT_phot', hue='GT_z',size='GT_phot',  palette='coolwarm', alpha=0.5)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [139]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
axs[0].scatter(pred_z, pred_ph, alpha=0.2)
axs[0].set_xlabel('Predicted z')
axs[0].set_ylabel('Predicted photon counts')

axs[1].scatter(gt_z, gt_ph, alpha=0.2)
axs[1].set_xlabel('Ground truth z')
axs[1].set_ylabel('Ground truth photon counts')


plt.show()

In [84]:
plt.figure()
plt.scatter(gt_z, gt_ph, alpha=0.2)
plt.xlabel('Ground truth z')
plt.ylabel('Ground truth photon counts')
plt.show()

In [73]:
plt.figure()
plt.scatter(gt_z, pred_z)
plt.xlabel('Ground truth z')
plt.ylabel('Prediction z')

Text(0, 0.5, 'Prediction z')

In [74]:
plt.figure()
plt.scatter(gt_ph, pred_ph)
plt.xlabel('Ground truth photon counts')
plt.ylabel('Prediction photon counts')

Text(0, 0.5, 'Prediction photon counts')

In [75]:
plt.figure()
plt.hist(gt_z - pred_z, bins=np.arange(-100, 100, 5))
plt.show()

In [76]:
plt.figure()
plt.hist(sampled_z, bins=np.arange(-500, 500, 10))
plt.show()

#### Predicted offsets

In [182]:
offsets_x = np.array([x[11] for x in preds_tmp])
offsets_y = np.array([x[12] for x in preds_tmp])
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].hist(offsets_x, bins=100,density=True)
ax[0].axvspan(offsets_x.min(), offsets_x.max(), color='green', alpha=0.1)
ax[0].set_xlabel('offsets_x')
ax[0].set_ylabel('counts')
ax[0].set_title(f"Mean offset_x is {np.mean(offsets_x):0.3f}, min:{np.min(offsets_x):0.3f}, max:{np.max(offsets_x):0.3f}")

ax[1].hist(offsets_y, bins=100, density=True)
ax[1].axvspan(offsets_y.min(), offsets_y.max(), color='green', alpha=0.1)
ax[1].set_xlabel('offsets_y')
ax[1].set_ylabel('counts')
ax[1].set_title(f"Mean offset_y is {np.mean(offsets_y):0.3f}, min:{np.min(offsets_y):0.3f}, max:{np.max(offsets_y):0.3f}")


fig.suptitle('Offsets predicted by the network')
plt.tight_layout()
plt.show()

In [183]:
gt_z.min(), gt_z.max()

type: name 'gt_z' is not defined

In [33]:
plt.figure()
plt.hist(gt_z, bins=np.arange(-500, 500, 20), label='ground truth', alpha=0.5)
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted', alpha=0.5)

plt.legend(loc='upper right')
plt.figure('')
plt.show()

In [34]:
plt.figure()
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted')
plt.legend(loc='upper right')
plt.show()

In [36]:
plt.figure()
plt.hist(gt_ph, bins=np.arange(700, 3000, 20), label='ground truth', alpha=0.4)
plt.hist(pred_ph, bins=np.arange(700, 3000, 20), label='predicted', alpha=0.4)

plt.legend(loc='upper right')
plt.show()

In [37]:
plt.figure()
plt.hist(sampled_ph, bins=np.arange(700, 3000, 10), label='sampled during training')
plt.show()

```
List of all matches localizations for further evaluation in format: 
    1. x_gt
    2. y_gt
    3. z_gt
    4. intensity_gt,
    5. x_pred,
    6. y_pred
    7. z_pred
    8. intensity_pred
    9. nms_p
    10.x_sig
    11.y_sig
    12.z_sig
```

In [38]:
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

In [39]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

fig.suptitle('Sigmas predicted by the network')
plt.tight_layout()
plt.show()

#### RMSE values distributions

In [40]:
residual_x = np.abs(gt_x - pred_x)
residual_y = np.abs(gt_y - pred_y)
residual_z = np.abs(gt_z - pred_z)
residual_ph = np.abs(gt_ph - pred_ph)

In [41]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(residual_x, bins=50)
ax[0, 0].axvspan(residual_x.min(), residual_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$Absoulte distance x $ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean {np.mean(residual_x):0.3f}, median: {np.median(residual_x):0.3f}, min:{np.min(residual_x):0.3f}, max:{np.max(residual_x):0.3f}")

ax[0, 1].hist(residual_y, bins=50)
ax[0, 1].axvspan(residual_y.min(), residual_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$Absoulte distance y $ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean dist y {np.mean(residual_y):0.3f}, median: {np.median(residual_y):0.3f}, min:{np.min(residual_y):0.3f}, max:{np.max(residual_y):0.3f}")

ax[1, 0].hist(residual_z, bins=50)
ax[1, 0].axvspan(residual_z.min(), residual_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$Absoulte distance z $ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean dist z {np.mean(residual_z):0.3f}, median: {np.median(residual_z):0.3f}, min:{np.min(residual_z):0.3f}, max:{np.max(residual_z):0.3f}")

ax[1, 1].hist(residual_ph, bins=50)
ax[1, 1].axvspan(residual_ph.min(), residual_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$Absoulte difference Photon counts $ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean diff photon counts {np.mean(residual_ph):0.3f}, median: {np.median(residual_ph):0.3f}, min:{np.min(residual_ph):0.3f}, max:{np.max(residual_ph):0.3f}")

fig.suptitle('Absolute distance btn prediction and ground truth')

plt.tight_layout()
plt.show()

### Real images

In [97]:
import skimage.io as sio

In [144]:
# 1-8 replisome focused, 9-16 replisome shifted, 17-24 origin focused, 25-32 origin shifted 
venus_filenames = []
for i in range(25, 27):
    venus_images = Path(f'/home/axtr7550/Chromosome_project/real_images/EXP-23-CA3051/Run/Pos{i}/fluor')
    venus_filenames.extend(sorted(list(venus_images.glob('*.tiff'))))

In [145]:
eval_imgs_all = []
for filename in venus_filenames:
    eval_imgs_all.append(sio.imread(filename))

: 

: 

: 

In [117]:
eval_imgs_all = np.stack(eval_imgs_all)
eval_imgs_all = eval_imgs_all[:,eval_imgs_all.shape[1]//2:,:]


In [118]:
plt.imshow(eval_imgs_all[50,...], cmap='gray', vmin = 80, vmax = 300)

In [120]:
eval2 = eval_imgs_all
fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
plot_num = None
preds_raw, n_per_img, plot_area = recognition(model=chromo_model, eval_imgs_all=eval2, batch_size=32, use_tqdm=False,
                                     nms=True, candidate_threshold=0.2,
                                     nms_threshold=0.5,
                                     pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                     plot_num=1,
                                     win_size=chromo_model.data_generator.simulation_params['train_size'],
                                     padding=True,
                                     padded_background=chromo_model.simulation_params['bg_values'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [121]:
df_real_raw_predictions = pd.DataFrame(preds_raw, columns=['id', 'image_id', 'x_pos', 'y_pos', 'z_pos', 'phot', \
    'prob', 'x_sigma', 'y_sigma', 'z_sigma', 'phot_sigma', 'x_offset', 'y_offset'])

df_real_raw_predictions['Shifted'] = True

In [122]:
df_all_predictions = df_real_raw_predictions.copy()

In [ ]:

df_all_predictions = pd.concat([df_all_predictions, df_real_raw_predictions], axis=0)


In [ ]:
df_all_predictions.to_csv('/home/axtr7550/Chromosome_project/real_images/EXP-23-CA3051/Run/all_predictions_17+25.csv')


In [ ]:
df_all_predictions 


,id,image_id,x_pos,y_pos,z_pos,phot,prob,x_sigma,y_sigma,z_sigma,phot_sigma,x_offset,y_offset,Shifted
0,1.0,1.0,27510.150391,5452.549316,156.578995,920.989685,0.884955,20.107000,19.061762,69.993690,79.881882,0.233087,-0.114626,False
1,2.0,1.0,29140.902344,5505.024414,154.516769,2273.683838,0.960698,10.615364,10.426609,33.502491,111.161400,0.321565,-0.307317,False
2,3.0,1.0,32005.203125,5669.065430,29.877623,1277.299438,0.919831,16.158756,16.014362,59.169415,93.492966,0.387734,0.216392,False
3,4.0,1.0,30045.466797,5774.012207,202.069214,1736.476807,0.982726,13.611437,12.713058,41.424801,116.240379,0.237964,-0.169045,False
4,6.0,1.0,32959.917969,5914.979492,238.493713,817.544434,0.685113,29.819738,26.880173,130.597778,40.624485,0.075686,-0.000316,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170960,5013.0,301.0,78411.359375,58952.859375,435.279938,1445.651367,0.558122,24.245485,23.571289,30.953636,175.554214,0.328630,-0.032949,False
170961,5014.0,301.0,76919.460938,59078.050781,-181.296936,877.212280,0.861102,21.466324,20.462055,71.420761,62.860424,0.376306,-0.106897,False
170962,5016.0,301.0,75592.921875,59294.808594,51.868347,861.979370,0.880185,23.592239,21.166479,81.390495,62.126534,-0.031932,0.227802,False
170963,5017.0,301.0,77430.953125,60055.679688,364.701172,986.925537,0.683422,24.408649,26.675735,66.856972,107.078018,0.245398,-0.066466,False


In [76]:
plt.close('all')


In [123]:
fig, ax = plt.subplots()
df_all_predictions[df_all_predictions['prob'] > 0.8]['z_pos'].hist(bins=100)
ax.set_xlabel('z')
ax.set_ylabel('count')

Text(0, 0.5, 'count')

In [124]:
fig, ax = plt.subplots()
df_all_predictions[df_all_predictions['prob'] > 0.8]['phot'].hist(bins=100)
ax.set_xlabel('phot')
ax.set_ylabel('count')

Text(0, 0.5, 'count')

In [112]:
plt.close('all')

In [47]:
df_real_filtered_preds = df_real_raw_predictions[df_real_raw_predictions['prob'] > 0.5]


In [5]:
real_path = '/home/axtr7550/Chromosome_project/real_images/EXP-23-CA3051/Run'
df_real_predictions = pd.read_csv(real_path + '/all_predictions_17+25.csv')


In [116]:
df_real_predictions = df_all_predictions.copy()


In [412]:
#There are a lot of quite low prob predictions in the bottom left corner of the haetmap.

df_bottom_left = df_real_predictions[(df_real_predictions['z_pos'] < -400) & (df_real_predictions['phot'] < 1000) & (df_real_predictions['prob'] < 0.6) & (df_real_predictions['Shifted']==True)]
df_bottom_left


,Unnamed: 0,id,image_id,x_pos,y_pos,z_pos,phot,prob,x_sigma,y_sigma,z_sigma,phot_sigma,x_offset,y_offset,Shifted,z_cat,phot_cat,prob_cat
377226,93,3.0,1.0,23730.507812,8754.356445,-471.586243,976.520752,0.553783,39.661404,37.727303,16.558777,129.310211,0.084725,-0.317590,True,-490.0,930.0,0.55
377314,181,47.0,1.0,39256.574219,15646.482422,-456.653961,878.613525,0.546617,38.297478,38.216492,26.478460,89.737122,-0.052694,-0.284880,True,-460.0,872.5,0.52
377445,312,13.0,1.0,26134.300781,18065.644531,-434.453094,980.088745,0.524476,41.177139,37.264328,36.081287,138.565063,0.066172,-0.067000,True,-460.0,930.0,0.52
377468,335,40.0,1.0,25146.613281,24086.638672,-437.662079,910.907410,0.551322,38.462128,38.132198,34.873100,104.005211,-0.129013,-0.436324,True,-460.0,872.5,0.55
377532,399,45.0,1.0,43494.402344,21984.529297,-449.829865,849.281494,0.520730,35.794399,34.884544,31.713423,70.613693,0.144636,0.223521,True,-460.0,815.0,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754383,377250,13204.0,301.0,65738.398438,28557.841797,-448.173096,822.223572,0.516110,40.903141,40.043167,28.951769,50.832634,0.359999,0.351425,True,-460.0,815.0,0.49
754524,377391,13998.0,301.0,49844.410156,39946.804688,-454.602051,826.777710,0.548798,39.222366,40.688595,26.941639,50.807369,-0.162949,-0.433771,True,-460.0,815.0,0.52
754783,377650,12560.0,301.0,32183.035156,57481.324219,-462.548615,839.178711,0.528942,45.212723,43.945503,24.466839,66.985168,0.123608,0.328090,True,-490.0,815.0,0.52
754876,377743,11068.0,301.0,67861.687500,49732.527344,-445.037354,829.527527,0.548123,41.682060,45.371635,41.794807,55.056168,0.025932,0.115802,True,-460.0,815.0,0.52


In [440]:
for index in np.random.randint(0, 5000, 10):
    row = df_bottom_left.iloc[index]

    img_id = int(row['image_id'])
    x_pos = int(row['x_pos']/65)
    y_pos = int(row['y_pos']/65)

    fig = plt.figure(figsize=(10, 10))

    offset = 20


    in_frame = df_real_predictions[(df_real_predictions['image_id'] == img_id) & (df_real_predictions['x_pos'] < (x_pos+offset)*65) &  (df_real_predictions['x_pos'] > (x_pos-offset)*65) & \
        (df_real_predictions['y_pos'] < (y_pos+offset)*65) &  (df_real_predictions['y_pos'] > (y_pos-offset)*65) & (df_real_predictions['Shifted'] == row['Shifted'])]

    plt.imshow(eval_imgs_all[img_id,  y_pos-offset:y_pos+offset, x_pos-offset:x_pos+offset], cmap='gray')

    plt.plot(in_frame['x_pos']/65 - x_pos + offset, in_frame['y_pos']/65 - y_pos + offset, 'o')

    for i in range(len(in_frame)):
        pl_row = in_frame.iloc[i]

        plt.text(pl_row['x_pos']/65 - x_pos + offset,  pl_row['y_pos']/65 - y_pos + offset, f"{pl_row['z_pos']:.0f}, {pl_row['prob']:.1f}, \n  {pl_row['phot']:.0f}", color='red')
    plt.title(f'{index}')



In [439]:
plt.close('all')

In [380]:
eval_imgs_all.shape



(301, 1041, 1302)

In [448]:

df_real_predictions['x_pos']//65 + df_real_predictions['x_offset']



0          371.972561
1          360.758878
2          400.997578
3          402.994953
4          435.903791
             ...     
755043    1053.768243
755044    1079.004716
755045    1050.040378
755046    1114.314422
755047    1125.143971
Length: 755048, dtype: float64

In [418]:
n_bins=25
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)
prob_bins = np.linspace(0.1,1.3, n_bins)


df_real_predictions['z_cat'] = None
df_real_predictions['phot_cat'] = None

df_real_predictions['prob_cat'] = None


for i in range(len(z_bins)-1):
    df_real_predictions.loc[(df_real_predictions['z_pos'] >= z_bins[i]) & (df_real_predictions['z_pos'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
    df_real_predictions.loc[(df_real_predictions['phot'] >= phot_bins[i]) & (df_real_predictions['phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]

    df_real_predictions.loc[(df_real_predictions['prob'] >= prob_bins[i]) & (df_real_predictions['prob'] <= prob_bins[i+1]),'prob_cat'] = prob_bins[i]



In [433]:
# Plot heatmap

labels_dict['prob'] = 'Probability'

p = 0.8
p_high = 3
shifted = False
data_df =df_real_predictions[(df_real_predictions['Shifted'] == shifted) & (df_real_predictions['prob'] > p) & (df_real_predictions['prob'] < (p_high))]


data, fig, ax = plot_heatmap_mean(df=data_df, x_col='z_cat', y_col='phot_cat', val_col='prob', precision = 0, count = True,extra_title='Shifted' if shifted else 'Not shifted',x_bins=z_bins, y_bins=phot_bins);
fig.suptitle(f'{"Shifted" if shifted else "Not Shifted"}, {p}< p < {p_high}')


Text(0.5, 0.98, 'Not Shifted, 0.8< p < 3')

In [434]:
fig, ax = plt.subplots()
y = list(data.sum(0))
plt.plot(z_bins,y)
fig.suptitle(f'{"Shifted" if shifted else "Not Shifted"}, {p}< p < {p_high}')


Text(0.5, 0.98, 'Not Shifted, 0.8< p < 3')

array([1.2644e+04, 1.6457e+04, 1.6780e+04, 1.4056e+04, 9.7700e+03,
       5.3540e+03, 2.3360e+03, 8.6800e+02, 2.6900e+02, 5.2000e+01,
       7.0000e+00, 1.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [435]:
plt.close('all')

In [119]:

df_not_shifted = df_real_predictions[df_real_predictions['Shifted']==False]
plot_heatmap_mean(df=df_not_shifted, x_col='z_cat', y_col='phot_cat', val_col='prob', precision = 0,count=True,extra_title='Not Shifted',x_bins=z_bins, y_bins=phot_bins);

In [228]:
path = '/home/axtr7550/Chromosome_project/real_images/' #C:/Users/Raket/OneDrive - Uppsala universitet/Desktop/Sommarjobb/Chromosome_project/data/'
df_matches = pd.read_csv(path+'rotated_matches.csv')
df_predictions = pd.read_csv(path+'rotated_predictions.csv')
df_GT = pd.read_csv(path+'rotated_GT.csv')


In [105]:
n_bins = 20

df_matches, z_bins, phot_bins = generate_cat_columns(df_matches, n_bins, z_bounds = [-700, 500], phot_bounds = [700, 3000])
df_matches.rename(columns={'Unnamed: 10':'phot_sigma'}, inplace=True)
df_matches


,GT_x,GT_y,GT_z,GT_phot,pred_x,pred_y,pred_z,pred_phot,Prob_nms,z_sigma,,,z_cat,phot_cat,GT_z_cat,GT_phot_cat,prob_cat
0,34717.208557,41470.444336,489.445806,1181.293666,34717.824219,41471.839844,188.382767,1252.852905,0.844369,59.477299,128.095276,0.120391,184.210526,1184.210526,436.842105,1063.157895,0.794737
1,60920.527954,6614.303436,-218.242556,2769.386709,60914.367188,6612.803223,-252.782700,2588.156738,0.851339,34.172104,224.206726,0.144111,-257.894737,2515.789474,-257.894737,2757.894737,0.794737
2,67081.047363,47392.863159,-184.122950,2955.434918,67074.539062,47391.050781,-194.572617,2960.127686,0.995758,22.319613,33.520729,-0.083985,-194.736842,2878.947368,-194.736842,2878.947368,0.984211
3,59736.071167,32705.186005,-126.408637,2928.610682,59738.464844,32698.390625,-124.891655,2739.081055,1.029449,36.411327,169.155914,0.053283,-131.578947,2636.842105,-131.578947,2878.947368,0.984211
4,46214.789734,15459.415283,90.423346,2802.913249,46211.105469,15467.000977,89.447372,2894.569092,0.908871,41.379623,79.670700,-0.059906,57.894737,2878.947368,57.894737,2757.894737,0.857895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20889,34973.784790,9393.775482,339.912653,1346.227169,35032.335938,9509.258789,98.996422,1156.426636,0.837261,72.012184,101.854286,-0.040979,57.894737,1063.157895,310.526316,1305.263158,0.794737
20890,50185.585938,55892.981873,464.011908,1034.962624,50324.707031,55915.390625,97.145569,1121.249756,0.889316,64.912071,93.080452,0.226255,57.894737,1063.157895,436.842105,942.105263,0.857895
20891,69234.934082,47466.638794,414.102256,1465.914756,69368.265625,47521.996094,20.939739,1096.740601,0.836145,69.901718,130.038330,0.204117,-5.263158,1063.157895,373.684211,1426.315789,0.794737
20892,23199.414062,47469.653931,127.164602,887.640774,23336.119141,47521.929688,14.330746,962.097900,0.890148,69.107910,97.116455,0.017228,-5.263158,942.105263,121.052632,821.052632,0.857895


In [107]:
labels_dict['Prob_nms'] = 'Probability '
plot_heatmap_mean(df=df_matches, x_col='z_cat', y_col='phot_cat', val_col='Prob_nms', precision = 0,count=True,extra_title='Not Shifted',x_bins=z_bins, y_bins=phot_bins);


784

In [139]:
n_bins = 20


z_bins = np.linspace(-700, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)


df_GT['GT_z_cat'] = None
df_GT['GT_phot_cat'] = None




for i in range(len(z_bins)-1):
    
    df_GT.loc[(df_GT['z'] >= z_bins[i]) & (df_GT['z'] <= z_bins[i+1]),'GT_z_cat'] = z_bins[i]
    df_GT.loc[(df_GT['phot'] >= phot_bins[i]) & (df_GT['phot'] <= phot_bins[i+1]),'GT_phot_cat'] = phot_bins[i]




df_GT


,emitter_index,image_index,y_position,x_position,z,phot,GT_z_cat,GT_phot_cat
0,1,1,26323.040161,7124.513664,-112.677127,923.177004,-131.578947,821.052632
1,2,1,30436.972046,7291.198158,-24.019867,1740.595937,-68.421053,1668.421053
2,3,1,25855.828857,7334.224358,-40.913999,2354.997754,-68.421053,2273.684211
3,4,1,26447.940216,7349.622383,-357.702553,2173.992991,-384.210526,2152.631579
4,5,1,32299.729462,7321.059418,-489.616424,2049.187303,-510.526316,2031.578947
...,...,...,...,...,...,...,...,...
21577,748,30,65244.714050,67219.346924,232.661247,2044.674397,184.210526,2031.578947
21578,749,30,66550.375366,67228.590698,-438.456595,2308.235943,-447.368421,2273.684211
21579,750,30,73299.251099,67237.263184,54.013252,1655.093014,-5.263158,1547.368421
21580,751,30,66302.023315,67295.439453,341.697812,1578.266680,310.526316,1547.368421


In [89]:
df_predictions



,counter of the molecule,image number,x,y,z,phot,prob_post_nms,x_sigma,y_sigma,z_sigma,phot_sigma,x_offset,y_offset
0,1.0,1.0,26253.763672,7193.437500,42.659542,989.330200,0.634554,27.281286,24.079681,95.932869,144.747620,-0.095942,-0.331734
1,2.0,1.0,30425.720703,7305.642578,-55.468636,1682.210327,0.993431,15.451195,14.255228,41.903572,125.016129,0.088004,0.394501
2,3.0,1.0,32285.841797,7289.623535,-470.247620,1901.780396,0.934352,14.452676,14.925750,17.720667,102.191261,-0.294750,0.148052
3,4.0,1.0,25847.394531,7334.836426,-51.805893,1962.256348,1.022542,12.511942,12.157673,37.591969,130.743256,-0.347762,-0.156364
4,5.0,1.0,26447.060547,7355.325684,-384.909424,2193.014404,1.049242,13.378056,14.072432,24.871910,157.546112,-0.122145,0.158854
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20636,267.0,30.0,66240.265625,67224.281250,323.345062,1194.787476,0.247438,22.614029,19.535463,74.570999,217.434586,0.081046,0.219734
20637,154.0,30.0,71708.085938,67091.031250,402.525787,2006.022339,0.806640,13.703774,13.905848,48.327641,118.984489,0.201378,0.169740
20638,155.0,30.0,67222.671875,67186.789062,225.227264,1030.423340,0.281296,18.861856,14.449130,72.709747,101.227783,0.194919,0.642892
20639,157.0,30.0,66565.945312,67182.062500,-430.269348,2229.870850,0.866132,12.972842,12.326520,24.624685,130.246063,0.091517,-0.429798


In [108]:
labels_dict['phot_sigma'] = 'phot_sigma'
counts_matches = plot_heatmap_mean(df=df_predictions, x_col='z_cat', y_col='phot_cat', val_col='phot_sigma', count=True, x_bins=z_bins, y_bins=phot_bins)

type: 'z_cat'

In [240]:

n_bins = 20


z_bins = np.linspace(-700, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)


df_predictions['z_cat'] = None
df_predictions['phot_cat'] = None




for i in range(len(z_bins)-1):
    
    df_predictions.loc[(df_predictions['z'] >= z_bins[i]) & (df_predictions['z'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
    df_predictions.loc[(df_predictions['phot'] >= phot_bins[i]) & (df_predictions['phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]




df_predictions


,counter of the molecule,image number,x,y,z,phot,prob_post_nms,x_sigma,y_sigma,z_sigma,phot_sigma,x_offset,y_offset,z_cat,phot_cat
0,1.0,1.0,27454.462891,6972.769043,75.515686,2627.178955,1.004784,11.086491,11.086788,40.509575,158.398865,0.376353,0.273373,57.894737,2515.789474
1,2.0,1.0,27086.564453,7110.162109,-666.124329,2621.364258,0.789499,10.624722,10.114514,17.944565,116.576508,-0.283634,0.387112,-700.0,2515.789474
2,1.0,1.0,33664.339844,7130.438965,-63.528324,2542.310547,1.038232,11.333193,10.937957,40.975258,137.179901,-0.087085,-0.300937,-68.421053,2515.789474
3,2.0,1.0,34083.199219,7222.888184,100.864281,1969.850830,1.031488,12.469643,12.232175,47.137516,121.800270,0.356904,0.121357,57.894737,1910.526316
4,3.0,1.0,39072.394531,7238.300293,-389.003967,2281.444336,1.042995,11.204569,11.520044,25.823814,106.955399,0.113740,0.358463,-447.368421,2273.684211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20743,173.0,30.0,69546.117188,66543.421875,268.827148,2399.643311,0.913802,15.300752,14.455275,41.412613,162.288986,-0.059747,-0.255108,247.368421,2394.736842
20744,175.0,30.0,70319.710938,66605.351562,136.702682,1453.484009,0.733231,18.893078,17.813614,72.969490,149.571808,-0.158257,-0.302238,121.052632,1426.315789
20745,176.0,30.0,72161.195312,66705.921875,-202.521423,817.066162,0.875347,24.110235,24.087296,66.814102,58.600685,0.172285,0.244979,-257.894737,700.0
20746,177.0,30.0,67421.937500,66751.218750,-436.597931,1561.467651,0.836017,21.015968,22.829243,50.638107,219.026260,0.260591,-0.058168,-447.368421,1547.368421


In [141]:
labels_dict['y_position'] = 'y'
counts_GT = plot_heatmap_mean(df=df_GT, x_col='GT_z_cat', y_col='GT_phot_cat', val_col='y_position', count=True)



type: Index(...) must be called with a collection of some kind, False was passed

In [142]:
missing_emitters = counts_GT - counts_matches
fig, ax = plt.subplots()
sns.heatmap(missing_emitters, annot=True)


type: name 'counts_GT' is not defined

In [392]:
plt.close('all')


In [173]:

prob_low = 0.5

df_prob_cap = df_matches[df_matches['Prob_nms'] > prob_low].copy()

In [174]:
# Create bins for drawing pdfs
n_bins = 41

z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,1700, n_bins)
prob_bins = np.linspace(0.1,1.3, n_bins)


df_prob_cap['z_cat'] = None
df_prob_cap['phot_cat'] = None

df_prob_cap['GT_z_cat'] = None
df_prob_cap['GT_phot_cat'] = None


df_prob_cap['prob_cat'] = None


for i in range(len(z_bins)-1):    
    df_prob_cap.loc[(df_prob_cap['GT_z'] >= z_bins[i]) & (df_prob_cap['GT_z'] <= z_bins[i+1]),'GT_z_cat'] = z_bins[i]
    df_prob_cap.loc[(df_prob_cap['GT_phot'] >= phot_bins[i]) & (df_prob_cap['GT_phot'] <= phot_bins[i+1]),'GT_phot_cat'] = phot_bins[i]



In [175]:
# Create pfds
from itertools import product
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)

all_pdfs = np.zeros((len(z_bins)-1, len(phot_bins)-1, len(z_bins)-1, len(phot_bins)-1))

squares = list(product(range(n_bins-1), range(n_bins-1)))

for i, j in squares:
    
    df_filtered = df_prob_cap.loc[(df_matches['GT_z'] >= z_bins[i]) & (df_prob_cap['GT_z'] <= z_bins[i+1]),:]
    df_filtered = df_filtered.loc[(df_filtered['GT_phot'] >= phot_bins[j]) & (df_filtered['GT_phot'] <= phot_bins[j+1]),:]


    hist, _, _ = np.histogram2d(df_filtered['pred_z'], df_filtered['pred_phot'], bins=(z_bins,phot_bins), density=True)

    all_pdfs[i,j,:,:] = hist


/home/axtr7550/anaconda3/envs/compile_spline/lib/python3.11/site-packages/numpy/lib/histograms.py:1065: RuntimeWarning: invalid value encountered in divide
  hist /= s


In [198]:

## Change GT distribution 
z_mean = 100
z_std = 200
phot_mean = 700
phot_std = 100
n_emitters = 10**5
z_values = np.random.normal(z_mean,  z_std, n_emitters)
phot_values = np.random.normal(phot_mean, phot_std, n_emitters)

in_range = (phot_values > 700) & (phot_values < 3000) & (z_values > -500) & (z_values < 500)
#phot_values = np.clip(phot_values, 701, 2999)
#z_values = np.clip(z_values, -499, 499)
z_values = z_values[in_range]
phot_values = phot_values[in_range]

z_indeces = (((z_values+500)*n_bins)/1000).astype(int)
phot_indeces = (((phot_values-700)*n_bins)/2300).astype(int)
indeces = np.stack((z_indeces, phot_indeces), axis=1)
indeces


array([[15,  0],
       [12,  1],
       [14,  1],
       ...,
       [19,  0],
       [14,  0],
       [11,  0]])

In [201]:
all_pdfs = np.nan_to_num(all_pdfs)

votes,_,_ = np.histogram2d(z_indeces, phot_indeces, bins =(np.arange(n_bins), np.arange(n_bins)))


In [202]:
# Make "predictions"

all_predictions = np.zeros((n_bins-1,n_bins-1))

for i, j in squares:
    n = int(votes[i,j])
    try: 
        vals = np.array(random.choices(np.arange((n_bins-1)**2), weights=all_pdfs[i,j,:,:].reshape((-1)), k=n))
    except ValueError:
        continue
    pred_votes, _, _ = np.histogram2d(vals//(n_bins-1), vals%(n_bins-1), bins = (np.arange(n_bins), np.arange(n_bins)))
    all_predictions += pred_votes


type: index 19 is out of bounds for axis 1 with size 19

In [205]:
# Plot fake heatmaps

heatmap_vals = np.flip(all_predictions.T, axis=0)

x_axis_vals = np.array(z_bins[:-1])
step = x_axis_vals[1] - x_axis_vals[0]
x_axis_vals = (x_axis_vals + step//2).astype(int)


y_axis_vals = np.array([val for val in phot_bins[-2::-1]])
step = y_axis_vals[1] - y_axis_vals[0]
y_axis_vals = (y_axis_vals - step//2).astype(int)


votes,_,_ = np.histogram2d(z_indeces, phot_indeces, bins =(np.arange(n_bins), np.arange(n_bins)))
fig, ax = plt.subplots()

sns.heatmap(np.flip(votes.T, axis=0), xticklabels = x_axis_vals,yticklabels=y_axis_vals, annot=True, fmt='.0f') 
ax.set_title(f"Ground truth, z mean={z_mean}, z std={z_std} \n  phot mean={phot_mean}, phot std={phot_std}, p > {prob_low}")


fig, ax = plt.subplots()

sns.heatmap(heatmap_vals,xticklabels = x_axis_vals,yticklabels=y_axis_vals, annot=True, fmt='.0f')
ax.set_title(f"Predictions, z mean={z_mean}, z std={z_std} \n  phot mean={phot_mean}, phot std={phot_std}, p > {prob_low}")



Text(0.5, 1.0, 'Predictions, z mean=100, z std=200 \n  phot mean=700, phot std=100, p > 0.5')

In [197]:
fig, ax = plt.subplots()
y = list(all_predictions.sum(1))
plt.plot(x_axis_vals, y)
plt.plot(x_axis_vals, votes.sum(1))
ax.set_title(f'Predictions distribution z  z mean={z_mean}, z std={z_std} \n  phot mean={phot_mean}, phot std={phot_std}, p > {prob_low}')

Text(0.5, 1.0, 'Predictions distribution z  z mean=100, z std=200 \n  phot mean=700, phot std=100, p > 0.5')

In [204]:
plt.close('all')

In [191]:
n_bins = 20
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)
prob_bins = np.linspace(0.5,1, n_bins)


df_all_predictions['z_cat'] = None
df_all_predictions['phot_cat'] = None


df_all_predictions['prob_cat'] = None


fig, ax = plt.subplots()
for i in range(len(z_bins)-1):
    df_all_predictions.loc[(df_all_predictions['z_pos'] >= z_bins[i]) & (df_all_predictions['z_pos'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
    df_all_predictions.loc[(df_all_predictions['phot'] >= phot_bins[i]) & (df_all_predictions['phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]
    

    df_all_predictions.loc[(df_all_predictions['prob'] >= prob_bins[i]) & (df_all_predictions['prob'] <= prob_bins[i+1]),'prob_cat'] = prob_bins[i]


type: name 'df_all_predictions' is not defined

In [192]:
labels_dict['prob'] = 'Predicted probability'
labels_dict['z_sigma'] = 'z sigma'
labels_dict['phot_sigma'] = 'z sigma'


data_df = df_all_predictions[df_all_predictions['Shifted']==False]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'phot_sigma', precision=3, extra_title='Not Shifted')


type: name 'df_all_predictions' is not defined

In [111]:
labels_dict['count'] = 'count'
data_df = df_all_predictions[df_all_predictions['Shifted']]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'z_sigma', count=True, extra_title='Shifted')

data_df = df_all_predictions[df_all_predictions['Shifted']==False]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'z_sigma', count=True, extra_title='Not shifted')



In [52]:
data_df = df_real_raw_predictions[(df_real_raw_predictions['phot'] >2400) & (df_real_raw_predictions['z_pos'] < 150) \
    &(df_real_raw_predictions['z_pos'] > -100)] 

In [53]:
fig, ax = plt.subplots()
sns.scatterplot(data=data_df, x="z_pos", y="phot", hue='z_sigma')

<Axes: xlabel='z_pos', ylabel='phot'>